In [1]:
#  *** THIS DATA (FROM API) HAS NOT BEEN CHECKED AGAINST DATA.CENSUS.GOV TABLES***

In [2]:
## Build an equitable economy: INCOME
### U.S. median income, Black median income, Black-white income gap
## 2015

In [3]:
# see https://jtleider.github.io/censusdata/api.html

In [4]:
# data source: 2015 Census Data (ACS-1 year) for Selected Variables - U.S.

In [5]:
# Dependencies
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from config import api_key

#c = Census(api_key)

In [6]:
import os

In [7]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis'

In [8]:
# need to change the working directory; be sure that the config file for the census API is also in this folder
#os.chdir('C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\analysis_nbs')

In [9]:
pd.set_option('display.max_rows', 200)

In [10]:
pd.set_option('max_colwidth', 100)

In [11]:
# see https://iqss.github.io/datafest-project/optional-u-s-census-data-python.html
# see census api example urls     https://api.census.gov/data/2019/acs/acs1/examples.html  
url = f"https://api.census.gov/data/2015/acs/acs1?get=NAME,B19001_001E,B01003_001E,B19013_001E,B02001_002E,B02001_003E,B19013A_001E,B19013B_001E&for=us:1&key={api_key}"

response = requests.get(url)
response.status_code

200

In [12]:
us = response.json()
us_df_income = pd.DataFrame(us[1:], columns=us[0])
# alternatively   us_df = pd.DataFrame(us[1:], columns=us[0]).rename(columns={'NAME':'country', 
# B01003_001E": "Population","B01002_001E": "Median Age"})
us_df_income 

,NAME,B19001_001E,B01003_001E,B19013_001E,B02001_002E,B02001_003E,B19013A_001E,B19013B_001E,us
0,United States,117259427,318857056,53657,233963128,40379066,57355,35481,1


In [13]:

us_df_income = us_df_income.rename(columns={"B19001_001E": "Total households",
                              "B01003_001E": "Population",
                              "B19013_001E": "Median household income",
                              "B02001_003E": "Pop. Black",
                              "B02001_002E": "Pop. white",
                              "B19013A_001E": "Median HH income White Alone Householder",
                              "B19013B_001E": "Median HH income Black or African American Alone Householder",  
                              "NAME": "Geography"})
us_df_income

,Geography,Total households,Population,Median household income,Pop. white,Pop. Black,Median HH income White Alone Householder,Median HH income Black or African American Alone Householder,us
0,United States,117259427,318857056,53657,233963128,40379066,57355,35481,1


In [14]:
us_df_income = us_df_income.copy()
us_df_income

,Geography,Total households,Population,Median household income,Pop. white,Pop. Black,Median HH income White Alone Householder,Median HH income Black or African American Alone Householder,us
0,United States,117259427,318857056,53657,233963128,40379066,57355,35481,1


In [15]:
# Export file as a CSV, without the Pandas index, but with the header
#us_df_income.to_csv("us_income_2015.csv", index = False, header=True)

In [ ]:
### DO NOT USE CELLS BELOW 

In [16]:
# data to be used to calculate poverty rate, labor participation rate, unemployment rate
url_rate = f"https://api.census.gov/data/2015/acs/acs1?get=NAME,B17001_001E,B17001_002E,B23025_001E,B23025_002E,B23025_004E,B23025_005E&for=us:1&key={api_key}"

response = requests.get(url_rate)
response.status_code

200

In [17]:
us_rates = response.json()
us_df_rates = pd.DataFrame(us_rates[1:], columns=us_rates[0])
# alternatively   us_df = pd.DataFrame(us[1:], columns=us[0]).rename(columns={'NAME':'country', 
# B01003_001E": "Population","B01002_001E": "Median Age"})
us_df_rates  

,NAME,B17001_001E,B17001_002E,B23025_001E,B23025_002E,B23025_004E,B23025_005E,us
0,United States,310899910,48208387,253588947,160532722,148019908,11530544,1


In [18]:
us_df_rates = us_df_rates.rename(columns={
                                      "B17001_001E": "Persons for whom poverty status determined",    
                                      "B17001_002E": "Individuals- income below poverty level",# universe: persons for whom poverty status is determined
                                      "B23025_001E":  "Population 16 years and over",
                                      "B23025_002E": "In labor force", # universe: Population 16 years and over
                                      "B23025_004E": "Employed civilians", # universe: Population 16 years and over
                                      "B23025_005E": "Unemployed civilians", # universe: Population 16 years and over
                                      "NAME": "Geography"
                                      })
us_df_rates.head()

,Geography,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,us
0,United States,310899910,48208387,253588947,160532722,148019908,11530544,1


In [19]:
# Add a new column for unemployment rate 
us_df_rates["Unemployment rate"] = (100 * \
    us_df_rates["Unemployed civilians"].astype(
        int) / us_df_rates["Population 16 years and over"].astype(int)).round()
us_df_rates.head()

,Geography,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,us,Unemployment rate
0,United States,310899910,48208387,253588947,160532722,148019908,11530544,1,5.0


In [20]:
# Add a new column for labor participation rate 
us_df_rates["Labor participation rate"] = (100 * \
    us_df_rates["In labor force"].astype(
        int) / us_df_rates["Population 16 years and over"].astype(int)).round(1)
us_df_rates.head()

,Geography,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,us,Unemployment rate,Labor participation rate
0,United States,310899910,48208387,253588947,160532722,148019908,11530544,1,5.0,-4.4


In [21]:
# Export file as a CSV, without the Pandas index, but with the header
#rates2015.to_csv("us_employment_2015.csv", index = False, header=True)